In [1]:
pip install imbalanced-learn

  Using cached scikit_learn-1.1.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30.8 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lale 0.6.10 requires scikit-learn<=1.0.2, but you have scikit-learn 1.1.3 which is incompatible.
autoai-ts-libs 1.1.9 requires scikit-learn<1.1,>=1.0.2; python_version >= "3.9", but you have scikit-learn 1.1.3 which is incompatible.
autoai-libs 1.13.6 requires scikit-learn<1.1,>=0.20.3; python_version >= "3.9", but you have scikit-learn 1.1.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-learn==1.0.2

  Using cached scikit_learn-1.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.4 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.3
    Uninstalling scikit-learn-1.1.3:
      Successfully uninstalled scikit-learn-1.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.9.1 requires scikit-learn>=1.1.0, but you have scikit-learn 1.0.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pickle-mixin

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import  GradientBoostingClassifier
%matplotlib inline


import warnings
warnings.filterwarnings('ignore')

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Yz-z7UF-8qT97BbdbpgWaY7pyFA-PyzCOsATWtR-Vzbo',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'smartlender-donotdelete-pr-sajhpdidcwjveh'
object_key = 'loan_prediction.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [6]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [7]:
numerical_features = df.select_dtypes(include = [np.number]).columns
categorical_features = df.select_dtypes(include = [np.object]).columns

In [8]:
df['Gender'] = df['Gender'].fillna(df['Gender'].mode()[0])
df['Married'] = df['Married'].fillna(df['Married'].mode()[0])
df['Dependents'] = df['Dependents'].str.replace('+','')
df['Dependents'] = df['Dependents'].fillna(df['Dependents'].mode()[0])
df['LoanAmount'] = df['LoanAmount'].fillna(df['LoanAmount'].mode()[0])
df['Self_Employed'] = df['Self_Employed'].fillna(df['Self_Employed'].mode()[0])
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0])
df['Credit_History'] = df['Credit_History'].fillna(df['Credit_History'].mode()[0])

In [9]:
df.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [10]:
df['Gender'].replace({'Male':1,'Female':0},inplace=True)
df['Dependents'].replace({'0':0,'1':1,'2':2,'3':3},inplace=True)
df['Married'].replace({'Yes':1,'No':0},inplace=True)
df['Self_Employed'].replace({'Yes':1,'No':0},inplace=True)
df['Property_Area'].replace({'Urban':2,'Rural':0,'Semiurban':1},inplace=True)
df['Education'].replace({'Graduate':1,'Not Graduate':0},inplace=True)
df['Loan_Status'].replace({'Y':1,'N':0},inplace=True)

In [11]:
df['CoapplicantIncome']=df['CoapplicantIncome'].astype("int64")
df['LoanAmount']=df['LoanAmount'].astype("int64")
df['Loan_Amount_Term']=df['Loan_Amount_Term'].astype("int64")
df['Credit_History']=df['Credit_History'].astype("int64")

In [12]:
le = LabelEncoder()
df['Loan_ID'] = le.fit_transform(df.Loan_ID)
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,1,0,0,1,0,5849,0,120,360,1,2,1
1,1,1,1,1,1,0,4583,1508,128,360,1,0,0
2,2,1,1,0,1,1,3000,0,66,360,1,2,1
3,3,1,1,0,0,0,2583,2358,120,360,1,2,1
4,4,1,0,0,1,0,6000,0,141,360,1,2,1


In [13]:
smote = SMOTETomek(0.90)

y = df['Loan_Status']
x = df.drop(columns=['Loan_Status'],axis=1)
x_bal,y_bal = smote.fit_resample(x,y)
print(y.value_counts())
print(y_bal.value_counts())

1    422
0    192
Name: Loan_Status, dtype: int64
1    361
0    318
Name: Loan_Status, dtype: int64


In [14]:
scaler = StandardScaler()
x_bal = scaler.fit_transform(x_bal)
x_bal = pd.DataFrame(x_bal)
x_bal.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-1.791352,0.566579,-1.208365,-0.707620,0.605142,-0.319801,0.105425,-0.571600,-0.301275,0.28614,0.580185,1.386456
1,-1.785548,0.566579,0.827565,0.336889,0.605142,-0.319801,-0.112428,-0.037052,-0.198762,0.28614,0.580185,-1.239495
2,-1.779744,0.566579,0.827565,-0.707620,0.605142,3.126944,-0.384831,-0.571600,-0.993242,0.28614,0.580185,1.386456
3,-1.773939,0.566579,0.827565,-0.707620,-1.652504,-0.319801,-0.456588,0.264252,-0.301275,0.28614,0.580185,1.386456
4,-1.768135,0.566579,-1.208365,-0.707620,0.605142,-0.319801,0.131409,-0.571600,-0.032177,0.28614,0.580185,1.386456


In [15]:
x_train, x_test, y_train, y_test = train_test_split(x_bal, y_bal, test_size = 0.33, random_state = 42)

# Compare the Models - KNN, Decision Tree, Random Forest CLassifier, XGBoost

# KNN

In [16]:
def kneighborsClassifier(x_train, x_test, y_train, y_test):
    knn = KNeighborsClassifier() 
    knn.fit(x_train,y_train)
    yPred = knn.predict(x_test)
    print("****KNN****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=knn.predict(x_test)
    y_pred1=knn.predict(x_train)
    knn_test_accuracy = accuracy_score(y_test,y_pred)
    knn_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ',knn_test_accuracy)
    print('Training accuracy: ', knn_train_accuracy)

kneighborsClassifier(x_train, x_test, y_train, y_test)

****KNN****
Confusion matrix
[[ 63  36]
 [ 25 101]]
Classification report
              precision    recall  f1-score   support

           0       0.72      0.64      0.67        99
           1       0.74      0.80      0.77       126

    accuracy                           0.73       225
   macro avg       0.73      0.72      0.72       225
weighted avg       0.73      0.73      0.73       225

Testing accuracy:  0.7288888888888889
Training accuracy:  0.8193832599118943


# Decision Tree

In [17]:
def decisionTreeClassifier(x_train, x_test, y_train, y_test):
    dt = DecisionTreeClassifier() 
    dt.fit(x_train,y_train)
    yPred = dt.predict(x_test)
    print("****DecisionTreeClassifier****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=dt.predict(x_test)
    y_pred1=dt.predict(x_train)
    decision_tree_test_accuracy = accuracy_score(y_test,y_pred)
    decision_tree_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ', decision_tree_test_accuracy)
    print('Training accuracy: ', decision_tree_train_accuracy)

decisionTreeClassifier(x_train, x_test, y_train, y_test)

****DecisionTreeClassifier****
Confusion matrix
[[79 20]
 [28 98]]
Classification report
              precision    recall  f1-score   support

           0       0.74      0.80      0.77        99
           1       0.83      0.78      0.80       126

    accuracy                           0.79       225
   macro avg       0.78      0.79      0.79       225
weighted avg       0.79      0.79      0.79       225

Testing accuracy:  0.7866666666666666
Training accuracy:  1.0


# Random Forest CLassifier

In [18]:
def randomForestClassifier(x_train, x_test, y_train, y_test):
    e = RandomForestClassifier() 
    rf = RandomForestClassifier() 
    rf.fit(x_train,y_train)
    yPred = rf.predict(x_test)
    print("****RandomForestClassifier****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=rf.predict(x_test)
    y_pred1=rf.predict(x_train)
    random_forest_test_accuracy = accuracy_score(y_test,y_pred)
    random_forest_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ', random_forest_test_accuracy)
    print('Training accuracy: ',random_forest_train_accuracy)

randomForestClassifier(x_train, x_test, y_train, y_test)

****RandomForestClassifier****
Confusion matrix
[[ 77  22]
 [ 14 112]]
Classification report
              precision    recall  f1-score   support

           0       0.85      0.78      0.81        99
           1       0.84      0.89      0.86       126

    accuracy                           0.84       225
   macro avg       0.84      0.83      0.84       225
weighted avg       0.84      0.84      0.84       225

Testing accuracy:  0.84
Training accuracy:  1.0


# XGBoost

In [19]:
def xgboost(x_train, x_test, y_train, y_test):
    xg = GradientBoostingClassifier() 
    xg.fit(x_train,y_train)
    yPred = xg.predict(x_test)
    print("****Gradient BoostingClassifier****") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=xg.predict(x_test)
    y_pred1=xg.predict(x_train)
    xgboost_test_accuracy = accuracy_score(y_test,y_pred)
    xgboost_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ', xgboost_test_accuracy)
    print('Training accuracy: ', xgboost_train_accuracy)
  
xgboost(x_train, x_test, y_train, y_test)

****Gradient BoostingClassifier****
Confusion matrix
[[ 66  33]
 [ 10 116]]
Classification report
              precision    recall  f1-score   support

           0       0.87      0.67      0.75        99
           1       0.78      0.92      0.84       126

    accuracy                           0.81       225
   macro avg       0.82      0.79      0.80       225
weighted avg       0.82      0.81      0.80       225

Testing accuracy:  0.8088888888888889
Training accuracy:  0.9229074889867841


In [20]:
kneighborsClassifier(x_train, x_test, y_train, y_test)
decisionTreeClassifier(x_train, x_test, y_train, y_test)
randomForestClassifier(x_train, x_test, y_train, y_test)
xgboost(x_train, x_test, y_train, y_test)

****KNN****
Confusion matrix
[[ 63  36]
 [ 25 101]]
Classification report
              precision    recall  f1-score   support

           0       0.72      0.64      0.67        99
           1       0.74      0.80      0.77       126

    accuracy                           0.73       225
   macro avg       0.73      0.72      0.72       225
weighted avg       0.73      0.73      0.73       225

Testing accuracy:  0.7288888888888889
Training accuracy:  0.8193832599118943
****DecisionTreeClassifier****
Confusion matrix
[[ 79  20]
 [ 24 102]]
Classification report
              precision    recall  f1-score   support

           0       0.77      0.80      0.78        99
           1       0.84      0.81      0.82       126

    accuracy                           0.80       225
   macro avg       0.80      0.80      0.80       225
weighted avg       0.81      0.80      0.80       225

Testing accuracy:  0.8044444444444444
Training accuracy:  1.0
****RandomForestClassifier****
Confusion

# Evaluating Performance Of The Model And Saving The Model

In [21]:
from sklearn.model_selection import cross_val_score
forest_reg = RandomForestClassifier(n_estimators = 10, criterion = "entropy", random_state = 42) 
forest_reg.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

In [22]:
yPred = forest_reg.predict(x_test)
y_pred=forest_reg.predict(x_test)
y_pred1=forest_reg.predict(x_train)
random_forest_test_accuracy = accuracy_score(y_test,y_pred)
random_forest_train_accuracy = accuracy_score(y_train,y_pred1)
print('Testing accuracy: ', random_forest_test_accuracy)
print('Training accuracy: ',random_forest_train_accuracy)
f1_score(yPred,y_test, average='weighted')
cv = cross_val_score(forest_reg,x,y,cv=5)
np.mean(cv)

Testing accuracy:  0.7911111111111111
Training accuracy:  0.9911894273127754


0.723057443689191

In [23]:
!pip install ibm_watson_machine_learning
!pip install watson_machine_learning_client

In [24]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url" : "https://us-south.ml.cloud.ibm.com",
    "apikey" : "Zci7zZ--GnCAaCBpuWwg0ubfosRAZfb25GbSUpBnFWLz"
}

wml_client=APIClient(wml_credentials)

In [25]:
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ------------  ------------------------
ID                                    NAME          CREATED
75dbbcd5-85fb-411f-a883-cd148d85c340  smart lender  2022-11-24T08:27:53.451Z
------------------------------------  ------------  ------------------------


In [26]:
space_id="75dbbcd5-85fb-411f-a883-cd148d85c340"

In [27]:
wml_client.set.default_space(space_id)

'SUCCESS'

In [28]:
import sklearn
sklearn.__version__

'1.0.2'

In [29]:
!pip install sklearn

In [30]:
wml_client.software_specifications.list(500)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [31]:
MODEL_NAME='Smart Lender'
DEPLOYMENT_NAME='smart lender'
LINEAR_MODEL=RandomForestClassifier()

In [32]:
#setup python version
software_spec_uid=wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [33]:
models_details=wml_client.repository.store_model(model=forest_reg,
    meta_props={
         wml_client.repository.ModelMetaNames.NAME:MODEL_NAME,
    wml_client.repository.ModelMetaNames.TYPE:'scikit-learn_1.0',
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
    })

Deployment creation failed for deployment id: 4c4c318c-66c5-456f-837a-ac707ed37fc6. Errors: [{'code': 'invalid_model_archive', 'message': 'Model loading through joblib failed with exception: KeyError and message: 0. The model object provided for deployment is either invalid or trained using a version of scikit-learn other than: 1.0', 'target': {'type': 'none', 'name': 'none'}, 'more_info': 'none'}]


In [35]:
models_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'scikit-learn_1.0'},
 'metadata': {'created_at': '2022-11-24T17:03:15.184Z',
  'id': 'a6134a8b-92aa-481f-a5d0-93e4a6c0d827',
  'modified_at': '2022-11-24T17:03:17.470Z',
  'name': 'Smart Lender',
  'owner': 'IBMid-6630042PTC',
  'resource_key': '5b1e0533-dbad-45db-97db-5debf289080a',
  'space_id': '75dbbcd5-85fb-411f-a883-cd148d85c340'},
 'system': {'warnings': []}}

In [38]:
models_id = wml_client.repository.get_model_id(models_details)
models_id

'a6134a8b-92aa-481f-a5d0-93e4a6c0d827'